In [5]:
import seaborn as sns
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
import string
from warnings import filterwarnings
filterwarnings('ignore')

In [6]:
df = pd.read_csv(r'E:\Python\ML\spam.csv', encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:
df.shape

(5572, 5)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [9]:
df.isna().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [10]:
df.duplicated().sum()

np.int64(403)

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace=True, axis=1)

In [13]:
df['v1'].replace({'spam':1,'ham':0}, inplace=True)

In [14]:
df['v1'].value_counts()

v1
0    4516
1     653
Name: count, dtype: int64

In [15]:
df.describe()

,v1
count,5169.000000
mean,0.126330
std,0.332253
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [16]:
df['v2'] = df['v2'].str.lower()

In [17]:
df.head()

,v1,v2
0,0,"go until jurong point, crazy.. available only ..."
1,0,ok lar... joking wif u oni...
2,1,free entry in 2 a wkly comp to win fa cup fina...
3,0,u dun say so early hor... u c already then say...
4,0,"nah i don't think he goes to usf, he lives aro..."


In [18]:
# Create a translation table that maps each punctuation character to None
translator = str.maketrans('', '', string.punctuation)

# Apply the translation to the 'message' column
df['v2'] = df['v2'].str.translate(translator)

In [19]:
import nltk
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
def prepocessing(sentence):
  words=sentence.split()
  filtered_tokens=[word for word in words if word not in stop_words]
  return ' '.join(filtered_tokens)

In [21]:
df['v2_preprocessed'] = df['v2'].apply(prepocessing)

In [22]:
df.head()

,v1,v2,v2_preprocessed
0,0,go until jurong point crazy available only in ...,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni,ok lar joking wif u oni
2,1,free entry in 2 a wkly comp to win fa cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,u dun say so early hor u c already then say,u dun say early hor u c already say
4,0,nah i dont think he goes to usf he lives aroun...,nah dont think goes usf lives around though


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
# tfdif_matrix= vectorizer.fit_transform(df['v2_preprocessed'])
# tfidf_array=tfdif_matrix.toarray()
# tfidf_array

In [24]:
X = df['v2_preprocessed']
Y= df['v1']
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix

vectorizer = TfidfVectorizer()
X_train_pre = vectorizer.fit_transform(X_train)
X_test_pre = vectorizer.transform(X_test)
clf = MultinomialNB()
clf.fit(X_train_pre,Y_train)

y_pred = clf.predict(X_test_pre)
print(accuracy_score(Y_test,y_pred))
print(confusion_matrix(Y_test,y_pred))
print(classification_report(Y_test,y_pred))

0.960348162475822
[[901   0]
 [ 41  92]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       901
           1       1.00      0.69      0.82       133

    accuracy                           0.96      1034
   macro avg       0.98      0.85      0.90      1034
weighted avg       0.96      0.96      0.96      1034



In [26]:
# clf.predict